In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import logging

from alea.sim.spacecraft.eps.power_system import PowerSystemModel
from alea.sim.spacecraft.eps.solar_panel import SolarPanelModel
from alea.sim.spacecraft.eps.eps import EPSModel


logging.basicConfig(level=logging.INFO)

from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel
from alea.sim.epa.earth_magnetic_field import EarthMagneticFieldModel
from alea.sim.kernel.kernel import AleasimKernel
from alea.sim.epa.disturbance_model import DisturbanceModel
from alea.sim.math_lib import Quaternion

In [ ]:
%load_ext autoreload
%autoreload 2

sim_dt=1e-3 #s
duration=10.0 #s
kernel = AleasimKernel(dt=sim_dt, date=2024.2)

adyn = AttitudeDynamicsModel(kernel)
odyn = OrbitDynamicsModel(kernel)
magm = EarthMagneticFieldModel(kernel)
DModel = DisturbanceModel(kernel)

power_sys = PowerSystemModel(kernel)
power_sys.eps._force_state_of_charge(0)

kernel.add_model(power_sys)
kernel.add_model(magm)
kernel.add_model(odyn)
kernel.add_model(adyn)
kernel.add_model(DModel)

solar_panels: list[SolarPanelModel] = power_sys.solar_panels
eps = power_sys.eps

kernel.set_log_level_all(logging.INFO)
adyn._state[0:4] = Quaternion.random().to_array()
adyn._state[4:7] = np.array([5, 10, 1]) * np.pi/180 #deg/s

kernel.advance(duration)

kernel.kill()

In [ ]:
#solar panel states - ['pwr_gen','energy_gen_total']
for panel in solar_panels:
    plt.plot(panel.time_array, panel.state_array[:,0], label=panel.name)
plt.legend()
plt.title('Power Generated per Panel')
plt.ylabel('Power (W)')
plt.xlabel('Time (s)')
plt.show()

#eps states: ['battery_soc_percent', 'battery_soc_Ah', 'watt_hour_estimate', 'solar_current', 'load_current', 'eps_voltage', 'power_net', 'power_in', 'power_out']
plt.plot(eps.time_array, eps.state_array[:,0])
plt.title('Eps State of Charge %')
plt.ylabel('%')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,1])
plt.title('Eps State of Charge Amp Hours')
plt.ylabel('Ah')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,2])
plt.title('Eps Watt Hours (inst.)')
plt.ylabel('Ah')
plt.xlabel('Time (s)')
plt.show()

objs = plt.plot(eps.time_array, eps.state_array[:,3:5])
plt.legend(iter(objs), tuple(eps.saved_state_element_names[3:5]))
plt.title('Currents')
plt.ylabel('A')
plt.xlabel('Time (s)')
plt.show()

plt.plot(eps.time_array, eps.state_array[:,5])
plt.title('Eps voltage')
plt.ylabel('V')
plt.xlabel('Time (s)')
plt.show()

objs = plt.plot(eps.time_array, eps.state_array[:, 6:9])
mean_power_in = eps.state_array[:, 7].mean()
mean_power_out = eps.state_array[:, 8].mean()
plt.hlines(mean_power_in, eps.time_array[0], eps.time_array[-1], label='Average Power In')
plt.hlines(mean_power_out, eps.time_array[0], eps.time_array[-1], label='Average Power Out')
plt.title("EPS Power")
plt.legend()
plt.ylabel('W')
plt.xlabel('Time (s)')
plt.show()


